In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [2]:

from tensorflow import keras
import numpy as np
import json

from sklearn.model_selection import train_test_split
np.set_printoptions(precision=3, suppress=True)
import io
import os
import tensorflow as tf
import  random
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import keras as keras
print(tf.__version__)
from keras.applications import  Xception
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw
import random
import matplotlib.image as mpimg
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)
RESIZE_SIZE=224
RGB = True
EPOCHS = 8
BATCH_SIZE = 64
TEST_SIZE = 0.2
WINDOWS_DATA_FOLDER="/mnt/e/code/ai/data/tcg/v1/"
JSONS_FOLDER=WINDOWS_DATA_FOLDER+"json/"
IMAGES_FOLDER=WINDOWS_DATA_FOLDER+"images/"

RANDOM_STATE=42

MIN_ASSETS=0
MAX_ASSETS=500
if RGB:
    RGB_VALUE = 3
else:
    RGB_VALUE = 1




2.19.0
2.19.0
Num GPUs Available:  0


In [ ]:

def load_files_list():

    json_files_list = []
    for json_file in os.listdir(JSONS_FOLDER):
        if json_file.endswith(".json"):
            json_files_list.append(json_file)
           

    random.Random(RANDOM_STATE).shuffle(json_files_list)
    json_files_list = json_files_list[MIN_ASSETS:MAX_ASSETS]
    print("Total files : ",json_files_list.__len__())
    print("First file : ",json_files_list[0])
    return json_files_list



In [ ]:
def read_json_from_s3(json_key):
    with open(JSONS_FOLDER + json_key) as json_data:
        raw_json = json.load(json_data)
        image_key = IMAGES_FOLDER + raw_json["asset"]["name"]
        ratio_x = RESIZE_SIZE / raw_json["asset"]["size"]["width"]
        ratio_y = RESIZE_SIZE / raw_json["asset"]["size"]["height"]
        points = raw_json["regions"][0]["points"]
        x0 = round(points[0]["x"] * ratio_x / RESIZE_SIZE,4)
        y0 = round(points[0]["y"] * ratio_y / RESIZE_SIZE,4)
        x1 = round(points[1]["x"] * ratio_x / RESIZE_SIZE,4)
        y1 = round(points[1]["y"] * ratio_y / RESIZE_SIZE,4)
        x2 = round(points[2]["x"] * ratio_x / RESIZE_SIZE,4)
        y2 = round(points[2]["y"] * ratio_y / RESIZE_SIZE,4)
        x3 = round(points[3]["x"] * ratio_x / RESIZE_SIZE,4)
        y3 = round(points[3]["y"] * ratio_y / RESIZE_SIZE,4)
        new_json = {
            "image_key": image_key,
            "bounding_boxes": [x0, y0, x1, y1, x2, y2, x3, y3]
        }
        return new_json
    return None
# Fonction pour lire les images depuis S3
def read_image_from_s3(image_key):
    try:
        img = Image.open(image_key)
        img = img.resize((RESIZE_SIZE, RESIZE_SIZE))
        img = np.asarray( img, dtype="int32" )
        np.array(img)
        img = img/255
        return img
    except Exception as e:
        print(e)
        return None
# Fonction pour créer un dataset TensorFlow
def create_dataset(json_keys, batch_size=32):
    def generator():
        for json_key in json_keys:
            annotations = read_json_from_s3(json_key)
            if annotations == None or annotations["image_key"] == None :
                print("File : " + json_key + " is Malformated, delete it")
                continue
            img = read_image_from_s3(annotations["image_key"])
            if img is None:
                print("Image : " + annotations["image_key"] + " is Malformated, delete it")
                continue
            yield img, annotations['bounding_boxes']
    dataset = tf.data.Dataset.from_generator(generator,
                                             output_signature=(
                                                 tf.TensorSpec(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE), dtype=tf.float32),
                                                 tf.TensorSpec(shape=(8,), dtype=tf.float32)
                                             ))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

In [ ]:
json_files_list = load_files_list()

train_files, val_files = train_test_split(json_files_list, shuffle=True, random_state=42, test_size=TEST_SIZE)
print("Train files : ", len(train_files))
print("Val files : ", len(val_files))
train_dataset = create_dataset(train_files, BATCH_SIZE)
val_dataset = create_dataset(val_files, BATCH_SIZE)


#read_image_from_s3(IMAGES_FOLDER+"ex9-67_-10_10_14_378_158.png")

In [ ]:

# Définir explicitement la couche d'entrée
input_layer = layers.Input(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE))
xception = Xception(weights="imagenet", include_top=False)(input_layer)

# Construire le reste du modèle
flatten = layers.Flatten(name='flatten')(xception)
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(flatten)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(8, activation='sigmoid', name='bb_head')(locator_branch)

# Créer le modèle
model = tf.keras.Model(inputs=input_layer, outputs=[locator_branch])

# Résumé du modèle
model.summary()

# Définir les pertes et compiler le modèle
losses = {"bb_head": tf.keras.losses.MeanSquaredError()}


model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])




In [ ]:
# Callbacks
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/model_checkpoint_{epoch:02d}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=False,
    verbose=1
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    mode='min',
    verbose=1
)

In [ ]:

# Entraînement du modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint,early_stopping]
)


In [ ]:
model.save("model-28k.keras")